In [ ]:
# import sys, os

# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# sys.path.append("..")
# from copy import copy, deepcopy

from avapi.carla import CarlaScenesManager


data_dir = "/data/shared/CARLA/multi-agent-random/"
CSM = CarlaScenesManager(data_dir, verbose=False)

## Visualizations

In [ ]:
from avapi.visualize.snapshot import show_image_with_boxes, show_lidar_bev_with_boxes


# Get scene dataset
CSD = CSM.get_scene_dataset_by_index(0)
frame_idx = 10
agent = 0
frame = CSD.get_frames(sensor="camera-0", agent=agent)[frame_idx]
all_cameras = ["camera-0"]
lidar = "lidar-0"

# loop over available sensors
for cam in all_cameras:
    if "DEPTH" in cam:
        img = CSD.get_depthimage(frame=frame, sensor=cam, agent=agent)
    else:
        img = CSD.get_image(frame=frame, sensor=cam, agent=agent)
    calib = CSD.get_calibration(frame=frame, sensor=cam, agent=agent)
    ego = CSD.get_agent(frame=frame, agent=agent)
    pc = CSD.get_lidar(frame=frame, sensor=lidar, agent=agent)
    objects = CSD.get_objects(frame, sensor=lidar, agent=agent)

    # visualize the results
    show_image_with_boxes(img, objects, inline=True)
    show_lidar_bev_with_boxes(pc, boxes=objects, inline=True)

## Perception

#### 2D Perception

In [ ]:
from avstack.modules.perception.object2dfv import MMDetObjectDetector2D


M_carla = MMDetObjectDetector2D(
    model="fasterrcnn", threshold=0.5, dataset="carla-joint"
)
M_cityscapes = MMDetObjectDetector2D(
    model="fasterrcnn", threshold=0.5, dataset="cityscapes"
)

In [ ]:
# run inference
cam = "camera-0"
img = CSD.get_image(frame=frame, sensor=cam, agent=agent)
dets_carla = M_carla(img)
dets_cityscapes = M_cityscapes(img)

# visualize
show_image_with_boxes(img, dets_carla, inline=True)
show_image_with_boxes(img, dets_cityscapes, inline=True)